# Test WANDB

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("mnm-shortlab-mit/allegro_hpo_vcrtiwzr")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv("project.csv")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/myless/.netrc
wandb: Currently logged in as: myless (mnm-shortlab-mit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
import pandas as pd
import ast
import numpy as np

# Read the CSV
df = pd.read_csv("project.csv")

# Function to safely extract metrics from the summary string
def extract_metrics(summary_str):
    try:
        summary_dict = ast.literal_eval(summary_str)
        runtime = summary_dict['_wandb']['runtime']
        val_rmse = summary_dict['validation_f_rmse']
        return runtime, val_rmse
    except:
        return None, None

# Extract metrics into new columns
df['runtime'], df['val_rmse'] = zip(*df['summary'].apply(extract_metrics))

# Remove rows with NaN values
df = df.dropna(subset=['runtime', 'val_rmse'])

# Convert runtime to hours
df['runtime_hours'] = df['runtime'] / 3600

# Function to find Pareto optimal points
def is_pareto_optimal(runtime, val_rmse):
    n_points = len(runtime)
    is_efficient = np.ones(n_points, dtype=bool)
    
    for i in range(n_points):
        for j in range(n_points):
            if i != j:
                if (runtime[j] <= runtime[i] and val_rmse[j] <= val_rmse[i] and 
                    (runtime[j] < runtime[i] or val_rmse[j] < val_rmse[i])):
                    is_efficient[i] = False
                    break
    
    return is_efficient

# Find Pareto optimal models
pareto_mask = is_pareto_optimal(df['runtime'].values, df['val_rmse'].values)
pareto_optimal = df[pareto_mask].sort_values('runtime')

# Display results
print("Found", len(pareto_optimal), "Pareto optimal models:")
for _, row in pareto_optimal.iterrows():
    print(f"\nModel: {row['name']}")
    print(f"Runtime: {row['runtime_hours']:.2f} hours")
    print(f"Validation RMSE: {row['val_rmse']:.4f}")

# Find best compromise model
# Normalize both metrics to 0-1 scale
runtime_norm = (pareto_optimal['runtime'] - pareto_optimal['runtime'].min()) / (pareto_optimal['runtime'].max() - pareto_optimal['runtime'].min())
rmse_norm = (pareto_optimal['val_rmse'] - pareto_optimal['val_rmse'].min()) / (pareto_optimal['val_rmse'].max() - pareto_optimal['val_rmse'].min())

# Calculate combined score (lower is better)
pareto_optimal['combined_score'] = runtime_norm + rmse_norm

# Get best compromise
best_compromise = pareto_optimal.loc[pareto_optimal['combined_score'].idxmin()]

print("\n\nBest Compromise Model:")
print(f"Model: {best_compromise['name']}")
print(f"Runtime: {best_compromise['runtime_hours']:.2f} hours")
print(f"Validation RMSE: {best_compromise['val_rmse']:.4f}")

# Get fastest and most accurate models for comparison
fastest = pareto_optimal.iloc[0]
most_accurate = pareto_optimal.loc[pareto_optimal['val_rmse'].idxmin()]

print("\nFastest Model:")
print(f"Model: {fastest['name']}")
print(f"Runtime: {fastest['runtime_hours']:.2f} hours")
print(f"Validation RMSE: {fastest['val_rmse']:.4f}")

print("\nMost Accurate Model:")
print(f"Model: {most_accurate['name']}")
print(f"Runtime: {most_accurate['runtime_hours']:.2f} hours")
print(f"Validation RMSE: {most_accurate['val_rmse']:.4f}")

Found 16 Pareto optimal models:

Model: allegro_model_0_r_max6p0_l_max0_num_layers1_num_tensor_features16_learning_rate0p0001
Runtime: 5.10 hours
Validation RMSE: 0.2842

Model: allegro_model_1_r_max6p0_l_max0_num_layers1_num_tensor_features16_learning_rate0p0005
Runtime: 5.19 hours
Validation RMSE: 0.2529

Model: allegro_model_2_r_max6p0_l_max0_num_layers1_num_tensor_features16_learning_rate0p001
Runtime: 5.24 hours
Validation RMSE: 0.2518

Model: allegro_model_4_r_max6p0_l_max0_num_layers1_num_tensor_features32_learning_rate0p0005
Runtime: 5.24 hours
Validation RMSE: 0.2480

Model: allegro_model_5_r_max6p0_l_max0_num_layers1_num_tensor_features32_learning_rate0p001
Runtime: 5.28 hours
Validation RMSE: 0.2440

Model: allegro_model_19_r_max6p0_l_max1_num_layers1_num_tensor_features16_learning_rate0p0005
Runtime: 5.30 hours
Validation RMSE: 0.2061

Model: allegro_model_20_r_max6p0_l_max1_num_layers1_num_tensor_features16_learning_rate0p001
Runtime: 5.35 hours
Validation RMSE: 0.2054

Mo

# Testing Elasticity

In [3]:
from forge.workflows.elasticity.lammps_elasticity import generate_lammps_elastic_input
from ase.io import read
from ase.build import bulk

#atoms = read('./scratch/data/mrs_neb_static/Cr4Ti7V111W4Zr2_14_to_15/00/POSCAR')
atoms = bulk('Fe', 'bcc', a=2.8665, cubic=True) * (4, 4, 4)
potential_file = '/Users/myless/Packages/mylammps/potentials/Fe_mm.eam.fs'
output_dir = './scratch/data/elasticity_test'
generate_lammps_elastic_input(atoms, potential_file, output_dir, job_name="elastic_calculation", use_gpu=True, use_kokkos=False)



[INFO] LAMMPS input files generated in: ./scratch/data/elasticity_test


# Database Working

In [1]:
from forge.core.database import DatabaseManager
from ase.io import read
import os

# 1. Your AWS database credentials
db_config = {
    'database': {
        'dbname': 'test_database',  # The name of the database you created on RDS
        'user': 'myless',           # Replace with your RDS username
        'password': 'vcrtiwzr',     # Replace with your RDS password
        'host': 'database-vcrtiwzr.cfg4i4qmuc4m.us-east-1.rds.amazonaws.com',
        'port': 5432                # Default Postgres port unless you changed it
    }
}

# 2. Instantiate the DatabaseManager
db_manager = DatabaseManager(config_dict=db_config)

# (Optional) If you need to drop and recreate tables, uncomment:
# with db_manager.conn.cursor() as cur:
#     cur.execute("""
#         DROP TABLE IF EXISTS calculations CASCADE;
#         DROP TABLE IF EXISTS structures CASCADE;
#     """)
# db_manager.conn.commit()
# db_manager._initialize_tables()

## Adding New Structures

In [10]:
from ase.io import read, write

data = read('./scratch/data/random_100_gen6.xyz', index=':')
#print(data[0])
#print(data[0].info)
#print(data[0].calc)
write('./scratch/data/random_100_gen6_fixed.xyz', data[5:])

In [12]:
db_manager.conn.rollback() # uncomment if your first add_structures_from_xyz fails

db_manager.add_structures_from_xyz(
    xyz_file='../data/gen_7-2025-02-12_og.xyz',
    skip_duplicates=True,
    default_model_type="vasp-static"
)

[INFO] Found 22201 frames in ../data/gen_7-2025-02-12_og.xyz
[INFO] Processing frame 0 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Skipping duplicate structure.
[INFO] Processing frame 1 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Added structure, ID=20197
[INFO] Added calculation, ID=20196
[INFO] Processing frame 2 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Added structure, ID=20198
[INFO] Added calculation, ID=20197
[INFO] Processing frame 3 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Added structure, ID=20199
[INFO] Added calculation, ID=20198
[INFO] Processing frame 4 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Added structure, ID=20200
[INFO] Added calculation, ID=20199
[INFO] Processing frame 5 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Added structure, ID=20201
[INFO] Added calculation, ID=20200
[INFO] Processing frame 6 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Added structure, ID=20202
[INFO] Added calculation, ID=20201
[INFO] Processing frame 7 from .

## Clearing Tables (Be Careful!)

In [2]:
# 0. Removing Duplicate Structures and their associated calculations
# Remove duplicates
duplicates = db_manager.remove_duplicate_structures(
    ltol=0.2,    # Length tolerance
    stol=0.3,    # Site tolerance
    angle_tol=5.0 # Angle tolerance in degrees
)

# Get a summary of what was removed
db_manager.get_duplicate_summary(duplicates)

Checking 162 structures for duplicates...
Found 1 duplicates of structure 1
Found 2 duplicates of structure 2
Found 1 duplicates of structure 3
Found 1 duplicates of structure 4
Found 1 duplicates of structure 5
Found 1 duplicates of structure 6
Found 1 duplicates of structure 7
Found 1 duplicates of structure 8
Found 6 duplicates of structure 9
Found 1 duplicates of structure 10
Found 1 duplicates of structure 11
Found 1 duplicates of structure 12
Found 1 duplicates of structure 13
Found 1 duplicates of structure 14
Found 1 duplicates of structure 15
Found 3 duplicates of structure 16
Found 2 duplicates of structure 17
Found 1 duplicates of structure 18
Found 1 duplicates of structure 19
Found 1 duplicates of structure 20
Found 1 duplicates of structure 21
Found 1 duplicates of structure 22
Found 2 duplicates of structure 23
Found 1 duplicates of structure 24
Found 1 duplicates of structure 25
Found 1 duplicates of structure 26
Found 1 duplicates of structure 27
Found 1 duplicates of 

In [4]:
# 1. Clear/Drop Tables
# -------------------
# BE CAREFUL: This will delete all data!
with db_manager.conn.cursor() as cur:
    cur.execute("""
        DROP TABLE IF EXISTS calculations CASCADE;
        DROP TABLE IF EXISTS structures CASCADE;
    """)
db_manager.conn.commit()
db_manager._initialize_tables()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


## Viewing Tables

In [2]:

# 2. View Tables
# -------------
# Count total structures and calculations
with db_manager.conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM structures")
    struct_count = cur.fetchone()[0]
    cur.execute("SELECT COUNT(*) FROM calculations")
    calc_count = cur.fetchone()[0]
print(f"Database contains {struct_count} structures and {calc_count} calculations")


Database contains 22087 structures and 22086 calculations


In [3]:

# View recent structures
with db_manager.conn.cursor() as cur:
    cur.execute("""
        SELECT structure_id, formula, composition, metadata->>'generation' as gen 
        FROM structures 
        ORDER BY structure_id DESC 
        LIMIT 5
    """)
    print("\nRecent structures:")
    for row in cur.fetchall():
        print(f"ID: {row[0]}, Formula: {row[1]}, Composition: {row[2]}, Generation: {row[3]}")




Recent structures:
ID: 22087, Formula: W252, Composition: {'W': {'at_frac': 1.0, 'num_atoms': 252}}, Generation: 6
ID: 22086, Formula: W252, Composition: {'W': {'at_frac': 1.0, 'num_atoms': 252}}, Generation: 6
ID: 22085, Formula: W252, Composition: {'W': {'at_frac': 1.0, 'num_atoms': 252}}, Generation: 6
ID: 22084, Formula: W252, Composition: {'W': {'at_frac': 1.0, 'num_atoms': 252}}, Generation: 6
ID: 22083, Formula: W53, Composition: {'W': {'at_frac': 1.0, 'num_atoms': 53}}, Generation: 6


In [4]:
# View calculations for a specific structure
db_manager.get_calculations(20191)

[{'id': 20191,
  'model_type': 'vasp-static',
  'energy': [-1034.10503071],
  'forces': array([[-9.229690e-01, -4.506000e-02,  1.344966e+00],
         [ 6.280400e-01,  2.901000e-03,  1.308524e+00],
         [-4.486000e-03,  1.293928e+00,  1.747160e-01],
         [-2.559973e+00,  1.277421e+00,  1.209620e-01],
         [ 6.437550e-01, -6.355900e-02, -5.342630e-01],
         [ 8.895380e-01, -3.517149e+00, -7.620430e-01],
         [-5.480000e-03,  4.239460e-01, -5.936040e-01],
         [ 1.559373e+00, -7.585400e-02, -9.632360e-01],
         [ 1.239175e+00, -7.066690e-01,  8.629400e-02],
         [-1.607800e-02,  8.207010e-01,  7.601330e-01],
         [ 1.130363e+00, -6.038600e-02, -8.498770e-01],
         [ 4.934040e-01,  2.622390e-01,  7.058600e-02],
         [ 8.845900e-02, -2.536150e-01,  1.254863e+00],
         [ 3.617710e-01,  3.201730e-01, -2.382980e-01],
         [-1.630334e+00, -6.509110e-01,  1.855767e+00],
         [-3.037850e-01, -2.719130e-01, -5.943230e-01],
         [-5.97950

## Search Examples

In [17]:
# 3. Search Examples
# ----------------
# Find structures with specific composition with debugging enabled
cr_structures = db_manager.find_structures(
    elements=['Cr'],  # Must contain Cr
    composition_constraints={
        'Cr': (0.1, 1.0)  # At least 1% Cr
    },
    debug=False  # Enable debugging
)

print(db_manager.get_structure(cr_structures[4]))
print(db_manager.get_calculations(cr_structures[4]))

Atoms(symbols='Cr25Ti25V24W25Zr25', pbc=True, cell=[[12.552320703, 0.033758657, -4.406501718], [-6.249853009, 10.8645436, -4.414950182], [0.001908008, -0.0071473, 13.224785071]])
[{'id': 16, 'model_type': 'vasp-static', 'energy': [-1113.77488505], 'forces': array([[-1.329357, -0.645809,  1.809289],
       [-0.191765, -2.395044,  0.301439],
       [ 0.405569,  0.890876,  5.05648 ],
       [ 0.03796 , -1.18591 , -0.895091],
       [ 0.169056,  2.336213,  0.369428],
       [ 1.167045,  0.076883,  0.661512],
       [-0.79992 ,  0.853109, -0.254597],
       [ 3.393114, -0.264754, -0.951341],
       [ 0.703927, -0.063254,  1.029836],
       [-1.047128,  0.833863, -2.079239],
       [ 0.336355, -0.739547,  0.062264],
       [ 2.439446,  1.178822,  1.689913],
       [ 0.405759,  1.055313, -0.808278],
       [ 1.721694,  0.997199,  0.689911],
       [-1.180354, -2.334016,  1.600422],
       [-1.089908,  2.214927, -0.593671],
       [ 0.91121 , -2.152509,  1.148381],
       [ 1.743501,  1.21594 

In [5]:
# Find structures with no calculations at all
no_calcs = db_manager.find_structures_without_calculation()

# Find structures without completed VASP calculations
no_vasp = db_manager.find_structures_without_calculation(model_type="vasp*", status="completed")

# Find structures without any VASP calculations (regardless of status)
no_vasp_any_status = db_manager.find_structures_without_calculation(model_type="vasp*", status=None)

print(f"Found {len(no_calcs)} structures with no calculations at all")
print(f"Found {len(no_vasp)} structures without completed VASP calculations")
print(f"Found {len(no_vasp_any_status)} structures without any VASP calculations (regardless of status)")

print(no_calcs)
print(db_manager.get_structure(no_calcs[0]))

Found 1 structures with no calculations at all
Found 1 structures without completed VASP calculations
Found 1 structures without any VASP calculations (regardless of status)
[20196]
Atoms(symbols='Cr117', pbc=True, cell=[8.535340639, 11.380454186, 14.225567732])


In [2]:
atoms = db_manager.get_atoms_with_calculation(structure_id=11245, model_type="vasp-static")
print(atoms)
print(atoms.info)

from ase.io import write
write('../data/test_atoms.xyz', atoms)


Atoms(symbols='Cr14Ti18V74W8Zr10', pbc=True, cell=[[12.612306025, -0.036812695, -4.453065183], [-6.303661977, 10.90479068, -4.356323681], [-0.012379756, 0.097350379, 13.323985214]], forces=...)
{'batch_id': True, 'generation': 0, 'composition': True, 'config_type': True, 'source_path': 'V-Cr-Ti-W-Zr/Fixed_Engrads/vcrtiwzr_fin_engrads/job_gen_0-2024-11-01/neb/structure_8238', 'temperature': True, 'date_processed': '2025-01-03T17:45:52.834447', 'structure_type': 'neb', 'structure_index': 8238, 'adversarial_step': True, 'energy': -1116.40199072, 'stress': array([ 1.20887359e-02,  6.27784095e-03,  8.11046662e-03,  1.12689199e-03,
       -7.21143964e-04,  5.04563598e-05]), 'calculation_id': 11245, 'model_type': 'vasp-static'}


In [13]:
# Find structures with high forces
high_force_structures = db_manager.find_structures(
    calculation_constraints={
        'model_type': 'vasp-static',
        'forces_max_magnitude': 0.5  # eV/Å
    }
)
print(f"\nFound {len(high_force_structures)} structures with forces >0.5 eV/Å")



TypeError: DatabaseManager.find_structures() got an unexpected keyword argument 'calculation_constraints'

In [14]:
# Find structures from a specific generation
gen5_structures = db_manager.find_structures(
    metadata_constraints={
        'generation': 5
    }
)
print(f"\nFound {len(gen5_structures)} structures from generation 5")

TypeError: DatabaseManager.find_structures() got an unexpected keyword argument 'metadata_constraints'

In [ ]:
# Combine multiple constraints
specific_structures = db_manager.find_structures(
    elements=['V', 'Cr'],
    composition_constraints={
        'V': (0.4, 0.6),    # 40-60% V
        'Cr': (0.1, 0.2)    # 10-20% Cr
    },
    metadata_constraints={
        'generation': 5,
        'structure_type': 'vac'
    },
    calculation_constraints={
        'model_type': 'vasp-static',
        'energy': (-1200, -1000)  # Energy range in eV
    }
)
print(f"\nFound {len(specific_structures)} structures matching all criteria")

## Examine Structure Details

In [ ]:

# 4. Examine Structure Details
# --------------------------
# Get full details for a specific structure
def examine_structure(db_manager, structure_id):
    """Get detailed information about a structure and its calculations."""
    with db_manager.conn.cursor() as cur:
        # Get structure details
        cur.execute("""
            SELECT formula, composition, metadata
            FROM structures
            WHERE structure_id = %s
        """, (structure_id,))
        row = cur.fetchone()
        if row:
            print(f"\nStructure {structure_id}:")
            print(f"Formula: {row[0]}")
            print(f"Composition: {row[1]}")
            print(f"Metadata: {row[2]}")
            
            # Get associated calculations
            cur.execute("""
                SELECT model_type, energy, stress, metadata
                FROM calculations
                WHERE structure_id = %s
            """, (structure_id,))
            print("\nCalculations:")
            for calc in cur.fetchall():
                print(f"- {calc[0]}: E={calc[1]:.3f} eV")
                if calc[2]:  # stress
                    print(f"  Stress tensor available")
                if calc[3]:  # metadata
                    status = calc[3].get('status', 'unknown')
                    print(f"  Status: {status}")
        else:
            print(f"No structure found with ID {structure_id}")

# Usage example:
if specific_structures:
    examine_structure(db_manager, specific_structures[0])